# Project 5: Camera Calibration and Fundamental Matrix Estimation with RANSAC
(adapted from the work developed by James Hays, Cusuh Ham, Arvind Krishnakumar, Jing Wu, John Lambert, Samarth Brahmbhatt, Grady Williams, and Henry Hu, which is originally based on a similar project by Aaron Bobick.)

  

## Overview
<font size="4">The goal of this programming assignment is to introduce you to camera and scene geometry. Specifically we will estimate the **camera projection matrix**, which maps 3D world coordinates to image coordinates, as well as the **fundamental matrix**, which relates points in one scene to epipolar lines in another. The camera projection matrix and fundamental matrix can each be estimated using point correspondences. To estimate the projection matrix (camera calibration), the input is corresponding 3D and 2D points. To estimate the fundamental matrix the input is corresponding 2D points across two images. You will start out by estimating the projection matrix and the fundamental matrix for a scene with ground truth correspondences. Then you will move on to estimating the fundamental matrix using point correspondences that are obtained using SIFT.</font>
 
<font size="4">Remember these challenging images of Gaudi’s Episcopal Palace from the programming assignment 2? By using RANSAC to find the fundamental matrix with the most inliers, we can filter away spurious matches and achieve near perfect point-to-point matching as shown below:</font>

<img src='meta_data/gaudi.png' height='1200'/>

<font size="4">For those of you who are passionate about **autonomous driving**, we will have a brief glimpse of the roles of epipolar geometry in this area. In practice, of course, it will be more complicated. For example, you need to deal with dynamic surroundings, different lighting condition, etc. But these fundamental geometric principles are still valuable and necessary.</font> 

## Details
<font size="4">In this programming assignment, you will work on:</font>
* <font size="4">Camera Projection Matrix  </font>
* <font size="4">Fundamental Matrix Estimation  </font>
* <font size="4">Fundamental Matrix with RANSAC   </font>
* <font size="4">Comparison of the results from previous steps</font>
* <font size="4">Using F-Matrix Estimation w/ RANSAC for Visual Odometry </font>

## Submission format
* <font size='4'>`<your_nu_username>.ipynb` (finished this file)
    
<font size='4'>**Note**: Do not install any additional packages inside the conda environment. The TAs will use the same environment as defined in the config files we provide you, so anything that’s not in there by default will probably cause your code to break during grading. Failure to follow any of these instructions will lead to point deductions. 

## Setup

In [ ]:
%matplotlib inline
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pa5_code.utils import (
    verify,
    evaluate_points,
    visualize_points,
    visualize_points_image,
    plot3dview,
    load_image,
    draw_epipolar_lines,
    get_matches,
    show_correspondence2
)

# Programming assignment starts here (100 points in total)

## Part 1: Camera Projection Matrix (25 points)

<font size="4" color="red">**task 1.1: Compute the projection from $[X,Y,Z,1]$ in homogenous coordinates to $(x,y)$ in non-homogenous image coordinates. (5 points).**</font><br><br>

In [ ]:
def projection(P: np.ndarray, points_3d: np.ndarray) -> np.ndarray:
    """
    Computes projection from [X,Y,Z,1] in homogenous coordinates to
    (x,y) in non-homogenous image coordinates.
    Args:
        P: 3 x 4 projection matrix
        points_3d: n x 4 array of points [X_i,Y_i,Z_i,1] in homogeneous
            coordinates or n x 3 array of points [X_i,Y_i,Z_i]
    Returns:
        projected_points_2d: n x 2 array of points in non-homogenous image
            coordinates
    """

    projected_points_2d = None
    ###########################################################################
    # TODO: YOUR CODE HERE                                                    #
    ###########################################################################
    
    raise NotImplementedError

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

    return projected_points_2d

In [ ]:
# let's check your implementation
from pa5_unit_tests.test_part1 import test_projection
print('projection():', verify(test_projection(projection)))
# test_projection(projection)

<font size="4" color="red">**task 1.2: Compute the projection matrix (20 points).**</font>

In [ ]:
def calculate_projection_matrix(
    points_2d: np.ndarray, points_3d: np.ndarray) -> np.ndarray:
    """
    To solve for the projection matrix. You need to set up a system of
    equations using the corresponding 2D and 3D points.

    Then you can solve this using least squares with np.linalg.lstsq() or SVD.
    Notice you obtain 2 equations for each corresponding 2D and 3D point
    pair. To solve this, you need at least 6 point pairs.

    Args:
        points_2d: A numpy array of shape (N, 2)
        points_2d: A numpy array of shape (N, 3)

    Returns:
        M: A numpy array of shape (3, 4) representing the projection matrix
    """
    
    M = None
    ###########################################################################
    # TODO: YOUR CODE HERE                                                    #
    ###########################################################################

    raise NotImplementedError    

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

    return M


In [ ]:
# let's check your implementation
from pa5_unit_tests.test_part1 import test_calculate_projection_matrix

print('calculate_projection_matrix():', verify(
    test_calculate_projection_matrix(calculate_projection_matrix)
))

## Now let's see how they work in practice
### Calculate the projection matrix given corresponding 2D & 3D points

In [ ]:
# Load the data
img_path = '../data/CCB_GaTech/pic_a.jpg'
points_2d = np.loadtxt('../data/CCB_GaTech/pts2d-norm-pic_a.txt')
points_3d = np.loadtxt('../data/CCB_GaTech/pts3d-norm.txt')
print('points2d: {}'.format(points_2d.shape))
print('points3d: {}'.format(points_3d.shape))

# (Optional) Uncomment these four lines once you have your code working with the easier, normalized points above.
# points_2d = np.loadtxt('../data/CCB_GaTech/pts2d-pic_b.txt')
# points_3d = np.loadtxt('../data/CCB_GaTech/pts3d.txt')

M = calculate_projection_matrix(points_2d, points_3d)
print('The projection matrix is\n', M)

[projected_2d_pts, residual] = evaluate_points(projection, M, points_2d, points_3d);
print('The total residual is {:f}'.format(residual))
plt.figure(); plt.imshow(load_image(img_path))
visualize_points(points_2d, projected_2d_pts)

## Camera Calibration for Argoverse image data
We'll now estimate the position of a camera mounted on an autonomous vehicle, using data from Argoverse. We'll use images from the "ring front center" camera, which faces forward.


<img src="https://www.argoverse.org/assets/images/reference_images/O2V4_vehicle_annotation.jpg" alt="Drawing" style="width: 400px;"/>


In [ ]:
# Argoverse Scene 3
img_path = '../data/argoverse_log_d60558d2_pair3/pic3.jpg'
points_2d = np.loadtxt('../data/argoverse_log_d60558d2_pair3/points_2d.txt')
points_3d = np.loadtxt('../data/argoverse_log_d60558d2_pair3/points_3d.txt')
# # # Argoverse Scene 2
# img_path = '../data/argoverse_log_d60558d2_pair2/pic2.jpg'
# points_2d = np.loadtxt('../data/argoverse_log_d60558d2_pair2/points_2d.txt')
# points_3d = np.loadtxt('../data/argoverse_log_d60558d2_pair2/points_3d.txt')

In [ ]:
M = calculate_projection_matrix(points_2d, points_3d)
print('The projection matrix is\n', M)

[projected_2d_pts, residual] = evaluate_points(projection, M, points_2d, points_3d);
print('The total residual is {:f}'.format(residual))
plt.figure(); plt.imshow(load_image(img_path))
visualize_points(points_2d, projected_2d_pts)

For these 2d-3d pairs, the "world" frame is defined as the "ego-vehicle" frame, where the origin is at the center of the back axle of the vehicle.

Thus, if your camera center estimate is correct, it should tell you how far to move forward (+x) and how far to move left (+y) and move up (+z) to reach teh camera's position.


The "egovehicle" coordinate system and "camera" coordinate system:
<img width="300"  src="https://user-images.githubusercontent.com/16724970/108759169-034e6180-751a-11eb-8a06-fbe344f1ee68.png">
<img width="300" src="https://user-images.githubusercontent.com/16724970/108759182-06495200-751a-11eb-8162-8b17f9cdee4b.png">

## Part 2: Fundamental Matrix Estimation (35 points)
<font size="4">We'll now solve for the Fundamental Matrix by implementing [Hartley's 8-Point algorithm](https://www.cse.unr.edu/~bebis/CS485/Handouts/hartley.pdf). We will use the normalized version.</font>

<font size="4" color="red">**task 2.1: Normalize the 2D coordinates (10 points).**</font>

<font size="4">As discussed in the lecture, you can use the following equation to normalize a 2D point.
<img src='meta_data/normalize_point.png' height=10/>
The transform matrix $T$ is the product of the scale and offset matrices. $c_u$ and $c_v$ are the mean coordinates that are to center the image data at the origin. To compute scales $s_u$ and $s_v$ you could estimate the standard deviation after subtracting the means. Then choose the scale factors $s_u$ and $s_v$ so the mean squared distance between the origin and the data points is 2 pixels.

In [ ]:
def normalize_points(points: np.ndarray) -> (np.ndarray, np.ndarray):
    """
    Perform coordinate normalization through linear transformations.
    Args:
        points: A numpy array of shape (N, 3) representing the 2D points in
            the image

    Returns:
        points_normalized: A numpy array of shape (N, 3) representing the
            normalized 2D points in the image
        T: transformation matrix representing the product of the scale and
            offset matrices
    """
    
    points_normalized, T = None, None
    ###########################################################################
    # TODO: YOUR CODE HERE                                                    #
    ###########################################################################

    raise NotImplementedError

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

    return points_normalized, T

In [ ]:
from pa5_unit_tests.test_part2 import test_normalize_points

print('test_normalize_points():', verify(
    test_normalize_points(normalize_points)
))

<font size="4" color="red">**task 2.2: Adjust the fundamental matrix to account for the normalized coordinates (5 points).**</font>
<img src='meta_data/denormalize_F.png' height=10/>

In [ ]:
def unnormalize_F(
    F_norm: np.ndarray, T_a: np.ndarray, T_b: np.ndarray) -> np.ndarray:
    """
    Adjusts F to account for normalized coordinates by using the transformation
    matrices.

    Args:
        F_norm: A numpy array of shape (3, 3) representing the normalized
            fundamental matrix
        T_a: Transformation matrix for image A
        T_B: Transformation matrix for image B

    Returns:
        F_orig: A numpy array of shape (3, 3) representing the original
            fundamental matrix
    """
    
    F_orig = None
    ###########################################################################
    # TODO: YOUR CODE HERE                                                    #
    ###########################################################################

    raise NotImplementedError

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

    return F_orig

In [ ]:
from pa5_unit_tests.test_part2 import test_unnormalize_F

print('test_unnormalize_F():', verify(
    test_unnormalize_F(unnormalize_F)
))

<font size="4" color="red">**task 2.3: Calculate the fundamental matrix with the normalized 8-point algorithm (20 points).**</font>

In [ ]:
def estimate_fundamental_matrix(
    points_a: np.ndarray, points_b: np.ndarray) -> np.ndarray:
    """
    Calculates the fundamental matrix. You may use the normalize_points() and
    unnormalize_F() functions here.

    Args:
        points_a: A numpy array of shape (N, 2) representing the 2D points in
            image A
        points_b: A numpy array of shape (N, 2) representing the 2D points in
            image B

    Returns:
        F: A numpy array of shape (3, 3) representing the fundamental matrix
    """
    
    F = None
    ###########################################################################
    # TODO: YOUR CODE HERE                                                    #
    ###########################################################################

    raise NotImplementedError

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

    return F


In [ ]:
from pa5_unit_tests.test_part2 import test_estimate_fundamental_matrix

print('test_estimate_fundamental_matrix():', verify(
    test_estimate_fundamental_matrix(estimate_fundamental_matrix)
))

### Estimate fundamental matrix with real-world data

In [ ]:
# Load the data
points_2d_pic_a = np.loadtxt('../data/CCB_GaTech/pts2d-pic_a.txt')
points_2d_pic_b = np.loadtxt('../data/CCB_GaTech/pts2d-pic_b.txt')
img_a = load_image('../data/CCB_GaTech/pic_a.jpg')
img_b = load_image('../data/CCB_GaTech/pic_b.jpg')

F = estimate_fundamental_matrix(points_2d_pic_a, points_2d_pic_b)

# Draw epipolar lines using the fundamental matrix
draw_epipolar_lines(F, img_a, img_b, points_2d_pic_a, points_2d_pic_b, figsize=(13,4))

## Part 3: Fundamental Matrix with RANSAC (30 points)

<font size="4">**Mount Rushmore**: This pair is easy, and most of the initial matches are correct. The base fundamental matrix estimation without coordinate normalization will work fine with RANSAC. 

<font size="4">**Notre Dame**: This pair is difficult because the keypoints are largely on the same plane. Still, even an inaccurate fundamental matrix can do a pretty good job of filtering spurious matches.  

<font size="4">**Gaudi**: This pair is difficult and doesn't find many correct matches unless you run at high resolution, but that will lead to tens of thousands of SIFT features, which will be somewhat slow to process. Normalizing the coordinates seems to make this pair work much better.  

<font size="4">**Woodruff**: This pair has a clearer relationship between the cameras (they are converging and have a wide baseline between them). The estimated fundamental matrix is less ambiguous and you should get epipolar lines qualitatively similar to part 2 of the project.

<font size="4" color="red">**task 3.1: Calculate the number of RANSAC iterations needed for a given guarantee of
    success (4 points).**</font>

In [ ]:
def calculate_num_ransac_iterations(
    prob_success: float, sample_size: int, ind_prob_correct: float) -> int:
    """
    Calculates the number of RANSAC iterations needed for a given guarantee of
    success.

    Args:
        prob_success: [float] representing the desired guarantee of success
        sample_size: [int] the number of samples included in each RANSAC
            iteration
        ind_prob_correct: [float] representing the probability that each element
            in a sample is correct

    Returns:
        num_samples: int the number of RANSAC iterations needed

    """
    num_samples = None
    ###########################################################################
    # TODO: YOUR CODE HERE                                                    #
    ###########################################################################

    raise NotImplementedError

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

    return int(num_samples)

In [ ]:
from pa5_unit_tests.test_part3 import test_calculate_num_ransac_iterations

print('test_calculate_num_ransac_iterations():', verify(
    test_calculate_num_ransac_iterations(calculate_num_ransac_iterations)
))

<font size="4" color="red">**task 3.2: Calculate the fundamental matrix with RANSAC. At most a single for loop can be used in your implementation (22 points).**</font><br>

In [ ]:
def ransac_fundamental_matrix(
    matches_a: np.ndarray, matches_b: np.ndarray) -> np.ndarray:
    """
    For this section, use RANSAC to find the best fundamental matrix by
    randomly sampling interest points. You would reuse
    estimate_fundamental_matrix() from part 2 of this assignment and
    calculate_num_ransac_iterations().

    If you are trying to produce an uncluttered visualization of epipolar
    lines, you may want to return no more than 30 points for either left or
    right images.

    Tips:
        0. You will need to determine your prob_success, sample_size, and
            ind_prob_success values. What is an acceptable rate of success? How
            many points do you want to sample? What is your estimate of the
            correspondence accuracy in your dataset?
        1. A potentially useful function is numpy.random.choice for creating
            your random samples.
        2. You will also need to choose an error threshold to separate your
            inliers from your outliers.
        3. Consider the geometric distances of a keypoint to its estimated 
           epipolar line. It is a bit more robust and the error threshold
           is easier to interpret (why?). Check the slide #70 in 
           cs5330-fall-2022-18.pptx.

    Args:
        matches_a: A numpy array of shape (N, 2) representing the coordinates
            of possibly matching points from image A
        matches_b: A numpy array of shape (N, 2) representing the coordinates
            of possibly matching points from image B
    Each row is a correspondence (e.g. row 42 of matches_a is a point that
    corresponds to row 42 of matches_b)

    Returns:
        best_F: A numpy array of shape (3, 3) representing the best fundamental
            matrix estimation
        inliers_a: A numpy array of shape (M, 2) representing the subset of
            corresponding points from image A that are inliers with respect to
            best_F
        inliers_b: A numpy array of shape (M, 2) representing the subset of
            corresponding points from image B that are inliers with respect to
            best_F
    
    
    """
    
    best_F, inliers_a, inliers_b = None, None, None
    ###########################################################################
    # TODO: YOUR CODE HERE                                                    #
    ###########################################################################

    raise NotImplementedError
    
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

    return best_F, inliers_a, inliers_b

In [ ]:
# let's check your implementation
from pa5_unit_tests.test_part3 import test_ransac_fundamental_matrix

print('test_ransac_fundamental_matrix():', verify(
    test_ransac_fundamental_matrix(ransac_fundamental_matrix)
))

### Let's work with real-world data

In [ ]:
# Load the data

# Woodruff
pic_a = load_image('../data/Woodruff_Dorm/wood1.jpg'); scale_a = 0.65
pic_b = load_image('../data/Woodruff_Dorm/wood2.jpg'); scale_b = 0.65
n_feat = 5e4

pic_a = cv2.resize(pic_a, None, fx=scale_a, fy=scale_a)
pic_b = cv2.resize(pic_b, None, fx=scale_b, fy=scale_b)

In [ ]:
# Finds matching points in the two images using OpenCV's implementation of SIFT.
# There can still be many spurious matches, though.
points_2d_pic_a, points_2d_pic_b = get_matches(pic_a, pic_b, n_feat)
print('Found {:d} possibly matching features'.format(len(points_2d_pic_a)))
match_image = show_correspondence2(pic_a, pic_b,
                                   points_2d_pic_a[:, 0], points_2d_pic_a[:, 1],
                                   points_2d_pic_b[:, 0], points_2d_pic_b[:, 1])
plt.figure(); plt.imshow(match_image)
plt.tight_layout()

### Calculate the Fundamental Matrix using RANSAC
<font size='4' color='red'>**Task 3.3: Get accurate matches and converging epipolar lines. No need to write any code. Simply run the code below (4 points).**</font>

In [ ]:
F, matched_points_a, matched_points_b = ransac_fundamental_matrix(points_2d_pic_a, points_2d_pic_b)

# Draw the epipolar lines on the images and corresponding matches
match_image = show_correspondence2(
    pic_a,
    pic_b,
    matched_points_a[:, 0], matched_points_a[:, 1],
    matched_points_b[:, 0], matched_points_b[:, 1]
)
plt.figure(); plt.imshow(match_image)
plt.tight_layout()

In [ ]:
draw_epipolar_lines(F, pic_a, pic_b, matched_points_a, matched_points_b, figsize=(12,8))

## Part 4: Visual Odometry (10 points)

For the following gif, we try to obtain the visual odometry of a camera mounted on a robot from the individual image frames.

![VO](https://user-images.githubusercontent.com/16724970/100487935-34fd8b00-30d9-11eb-9941-7735fcae445c.gif "VO")

<font size="4" color="Red">**Task 4.1: Convert a fundamental matrix to the essential matrix (3 points).**</font>

In [ ]:
def get_emat_from_fmat(F: np.ndarray, K1: np.ndarray, K2: np.ndarray) -> np.ndarray:
    """ 
    Create essential matrix from camera instrinsics and fundamental matrix
    Args:
        F:  A numpy array of shape (3, 3) representing the fundamental matrix between
            two cameras
        K1: A numpy array of shape (3, 3) representing the intrinsic matrix of the
            first camera
        K2: A numpy array of shape (3, 3) representing the intrinsic matrix of the
            second camera

    Returns:
        E:  A numpy array of shape (3, 3) representing the essential matrix between
            the two cameras.
    """
    
    E = None
    ###########################################################################
    # TODO: YOUR CODE HERE                                                    #
    ###########################################################################
    
    raise NotImplementedError

    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    
    return E

In [ ]:
# let's check your implementation
from pa5_unit_tests.test_part4 import test_get_emat_from_fmat

print('test_get_emat_from_fmat():', verify(
    test_get_emat_from_fmat(get_emat_from_fmat)
))

### Let's see check the visual odometry estimations on a real-world autonomous driving sequence

In [ ]:
from pa5_code.vo import get_visual_odometry, plot_poses, evaluate_poses

# This may take a few minutes to run across 20 image frames from the Argoverse dataset
poses_wTi = get_visual_odometry(get_emat_from_fmat, ransac_fundamental_matrix)

### Red dots denote ground-truth camera poses and green ones are the estimations

<font size='4' color='red'>**Task 4.2: Get good camera pose estimation. The plotted trajectories of ground-truth and estimated camera poses should roughly match. No need to write any code. Simply run the code below (3 points).**</font>

In [ ]:
poses_wTi_gt = np.load('../data/vo_seq_argoverse_273c1883/gt_poses.npy')
plot_poses(poses_wTi, poses_wTi_gt)
avg_r_err, avg_t_err = evaluate_poses(poses_wTi, poses_wTi_gt)
print('Average rotation error (in degree): {}'.format(avg_r_err))
print('Average angular error of translation (in degree): {}'.format(avg_t_err))

<font size='4' color='red'>**Task 4.3: Question: In addition to the fundamental matrix, what additional camera information is required to recover the ego-motion? (4 points)**</font>

[Your answer here]